<a href="https://colab.research.google.com/github/jacobdwatters/NIOSH-Project/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Setup

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

import scipy as sp
from scipy import stats

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load and Process Data

In [3]:
path_after_2010 = '/content/gdrive/My Drive/NIOSH Project/data/violations_processed_after_2010.csv'
violation_data = pd.read_csv(path_after_2010)

In [4]:
FEATURES = ['MINE_TYPE', 'COAL_METAL_IND', 'SIG_SUB', 'LIKELIHOOD', 
            'INJ_ILLNESS', 'NO_AFFECTED', 'NEGLIGENCE', 'VIOLATOR_VIOLATION_CNT',
            'VIOLATOR_INSPECTION_DAY_CNT']
TARGETS = ['PROPOSED_PENALTY']

X = violation_data[FEATURES]
y = violation_data[TARGETS]

In [5]:
# Define which columns should be encoded vs scaled
columns_to_encode = ['MINE_TYPE', 'COAL_METAL_IND', 'LIKELIHOOD', 'INJ_ILLNESS', 'SIG_SUB', 'NEGLIGENCE']
columns_to_scale  = ['VIOLATOR_VIOLATION_CNT', 'NO_AFFECTED', 'VIOLATOR_INSPECTION_DAY_CNT']

# Instantiate encoder/scaler
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False)

# Scale and Encode Separate Columns
scaled_columns  = scaler.fit_transform(X[columns_to_scale])
encoded_columns = ohe.fit_transform(X[columns_to_encode])

# Concatenate (Column-Bind) Processed Columns Back Together
X_pre = np.concatenate([scaled_columns, encoded_columns], axis=1)
np.nan_to_num(X_pre, copy=False)

print('Features shape:', X_pre.shape)

Features shape: (1429135, 24)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_pre, y, test_size = 0.25, random_state = 0)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_train shape:', y_test.shape)

X_train shape: (1071851, 24)
X_test shape: (357284, 24)
y_train shape: (1071851, 1)
y_train shape: (357284, 1)


## Train Model

In [10]:
regr = RandomForestRegressor(max_depth=15, random_state=0)
regr.fit(X_train, y_train)

r2 = regr.score(X_test, y_test)
print('R2 %.3f' % r2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


R2 0.588


## Prepare inlier Data

In [12]:
MAX_VALUE = 1000
violations_cliped = violation_data[violation_data['PROPOSED_PENALTY'] <= MAX_VALUE]

print('Total samples = %d' % len(violation_data))
print('Total samples with PROPOSED_PENALTY <= %d: %d' % (MAX_VALUE, len(violations_cliped)))

Total samples = 1429135
Total samples with PROPOSED_PENALTY <= 1000: 1259626


In [13]:
X_inliers = violations_cliped[FEATURES]
y_inliers = violations_cliped[TARGETS]

# Instantiate encoder/scaler
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False)

# Scale and Encode Separate Columns
scaled_columns_inliers  = scaler.fit_transform(X_inliers[columns_to_scale])
encoded_columns_inliers = ohe.fit_transform(X_inliers[columns_to_encode])

# Concatenate (Column-Bind) Processed Columns Back Together
X_pre_inliers = np.concatenate([scaled_columns_inliers, encoded_columns_inliers], axis=1)
np.nan_to_num(X_pre_inliers, copy=False)

X_train_inliers, X_test_inliers, y_train_inliers, y_test_inliers = train_test_split(X_pre_inliers, y_inliers, test_size = 0.25, random_state = 0)

## Inlier Model

In [14]:
regr_inliers = RandomForestRegressor(max_depth=15, random_state=0)
regr_inliers.fit(X_train_inliers, y_train_inliers)

r2_inlier = regr_inliers.score(X_test_inliers, y_test_inliers)
print('R2 %.3f' % r2_inlier)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


R2 0.735


## Prepare Outlier Data

In [15]:
violations_outliers = violation_data[violation_data['PROPOSED_PENALTY'] > MAX_VALUE]

print('Total samples = %d' % len(violation_data))
print('Total samples with PROPOSED_PENALTY > %d: %d' % (MAX_VALUE, len(violations_outliers)))

Total samples = 1429135
Total samples with PROPOSED_PENALTY > 1000: 169509


In [16]:
X_outliers = violations_outliers[FEATURES]
y_outliers = violations_outliers[TARGETS]

# Instantiate encoder/scaler
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False)

# Scale and Encode Separate Columns
scaled_columns_outliers  = scaler.fit_transform(X_outliers[columns_to_scale])
encoded_columns_outliers = ohe.fit_transform(X_outliers[columns_to_encode])

# Concatenate (Column-Bind) Processed Columns Back Together
X_pre_outliers = np.concatenate([scaled_columns_outliers, encoded_columns_outliers], axis=1)
np.nan_to_num(X_pre_outliers, copy=False)

X_train_outliers, X_test_outliers, y_train_outliers, y_test_outliers = train_test_split(X_pre_outliers, y_outliers, test_size = 0.25, random_state = 0)

## Outlier Model

In [19]:
regr_outliers = RandomForestRegressor(max_depth=10, random_state=42)
regr_outliers.fit(X_train_outliers, y_train_outliers)

r2_outliers = regr_inliers.score(X_test_outliers, y_test_outliers)
print('R2 %.3f' % r2_outliers)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


R2 -0.166
